In [3]:
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

/tmp/ipykernel_562313/462869928.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook as tqdm


In [4]:
df_predictions = pd.read_csv("test_dataset__bart_model_language_rating_prediction.csv")

In [5]:
df = pd.read_csv("kidsInMindSubtitles2004.csv")
df.shape

(2004, 15)

In [4]:
import torch
from transformers import pipeline

# use dolly-v2-12b if you're using Colab Pro+, using pythia-2.8b for Free Colab
generate_text = pipeline(model="databricks/dolly-v2-2-8b", 
                         torch_dtype=torch.bfloat16, 
                         trust_remote_code=True,
                         device_map="auto")

2024-02-07 18:57:56.209431: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-07 18:57:56.244698: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-07 18:57:56.880462: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
df_predictions['dolly_8b'] = None

In [21]:

for idx in tqdm(range(len(df_predictions))):
    
    query = "Given the subtitle rating of a movie = "+str(round(df_predictions['Predcited_Score'][idx], 1))+" \
    ; and a list of inappropriate words : "+ df_predictions['inappropriate_words'][idx] + " Generate \
    language summary like kids in mind website? Please consider the below two example language summaries of \
    Please consider the below two example language summaries of \
    movies:  \n 1." + df['Language_Description'][0] + "\n 2."+df['Language_Description'][1]
    
    query = query.replace("[", "")
    query = query.replace("]", "\n\n")
    
    
    response = generate_text(query)
    
    df_predictions['dolly_8b'][idx] = response[0]['generated_text']
    

  0%|          | 0/201 [00:00<?, ?it/s]

/tmp/ipykernel_509381/241509247.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['dolly_8b'][idx] = response[0]['generated_text']
/tmp/ipykernel_509381/241509247.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['dolly_8b'][idx] = response[0]['generated_text']
/tmp/ipykernel_509381/241509247.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['dolly_8b'][idx] = response[0]['generated

In [42]:
df_predictions.to_csv("df_predictions_dolly_8b.csv", index = False)

In [46]:
#df_predictions.head(2)

In [9]:
df_predictions = pd.read_csv("df_predictions_dolly_8b.csv")

In [36]:
df_predictions['dolly_8b'][1]

'At least 1 f-word, 4 scatological terms, 4 anatomical terms, 3 mild obscenities, name-calling (freak, psycho, foolish, boring, stubborn, little man), 4 religious exclamations.'

In [37]:
df_predictions['language_summary'][1]

'About 25 F-words, 2 obscene hand gestures, 8 sexual references, 10 scatological terms, 13 anatomical terms, 23 mild obscenities, name-calling (dummy, psycho, freak show, retarded, dumb, jerk, idiot, "are you deaf or stupid," ugly), 17 religious exclamations.\n'

In [26]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [28]:
scores = scorer.score(df_predictions['language_summary'][0], df_predictions['dolly_8b'][0])

In [29]:
scores

{'rouge1': Score(precision=0.4166666666666667, recall=0.2631578947368421, fmeasure=0.3225806451612903),
 'rouge2': Score(precision=0.2, recall=0.125, fmeasure=0.15384615384615385),
 'rougeL': Score(precision=0.3333333333333333, recall=0.21052631578947367, fmeasure=0.2580645161290323)}

In [30]:

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in range(len(df_predictions)):
    
    scores = scorer.score(df_predictions['language_summary'][idx], df_predictions['dolly_8b'][idx])
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)


In [33]:
sum(rougeL_scores)/len(rougeL_scores)

0.14377770163453993

In [7]:
import bert_score

In [10]:
df_predictions

,Language_Score,Predcited_Score,subtitle,language_summary,cleaned_text,inappropriate_words,inappropriate_words_count,dolly_8b
0,6,6.543292,The High Council\nhas reached a verdict --guil...,"13 F-words, 1 nearly uttered F-word, 3 obscene...",the high council reached verdict --guilty!you ...,"['bondage', 'bitch', 'ass', 'fuck', 'fuck', 'g...",47,Here are 2 example of generated summary. Pleas...
1,7,7.480484,"Danny, give me my shirt!Come and get it.Don't ...","About 25 F-words, 2 obscene hand gestures, 8 s...","danny, shirt!come it.don't think maybe we're t...","['douche', 'damn', 'bitch', 'bra', 'sex', 'lai...",103,"At least 1 f-word, 4 scatological terms, 4 ana..."
2,7,8.941082,BOYZ II MEN SINGING:\nYou taught me everything...,"About 28 F-words, a few racial slurs (African-...",boyz ii men singing: you taught everythingand ...,"['sex', 'butt', 'ass', 'fat', 'ass', 'church',...",25,Here is an example of how you can generate the...
3,9,9.959660,"INGLEWOOD,\nCALIFORNIA, 1979<i>I always loved ...","48 F-words and its derivatives, 27 sexual refe...","inglewood, california, 1979<i>i loved airplane...","['ass', 'ass', 'die', 'killed', 'ass', 'damn',...",118,Here is the original website: \nhttps://www.u...
4,1,1.016869,I was born in Hawaii to two die-hard surfers.H...,"Exclamations (oh my gosh, darn it), name-calli...",i born hawaii die-hard surfers.how i salt wate...,"['die', 'nuts', 'attack', 'harder', 'stringer'...",12,Please do not be alarmed! This is just a web p...
...,...,...,...,...,...,...,...,...
196,1,1.109995,[<b>Subtitles Added by Neeraj(neeraj6488@gmail...,2 religious exclamations.\n,[<b>subtitles added neeraj(neeraj6488@gmail.co...,"['hell', 'fuck', 'snatch', 'fucking', 'herpes'...",87,"movies: At least 1 F-word, 4 scatological term..."
197,1,2.468549,- Honey... did you hear that?\n- Hear what?Bar...,A few mild obscenities.\n,- honey... hear that? - hear what?barbara?did ...,"['hell', 'hell', 'urine', 'killing', 'ass', 'v...",11,F-word : at least 1\nScatological terms : 4\nA...
198,6,7.052896,"Oh, man. I can't get over\nhow different we lo...","About 10 F-words, some scatological terms, and...","oh, man. i can't different look.maybe it's uni...","['crime', 'killer', 'killed', 'goddamn', 'ass'...",61,Your question (Generating language summaries l...
199,6,6.756784,Come this way!Men wounded in battle we can dea...,"Nearly 10 F-words, several anatomical referenc...",come way!men wounded battle deal with.but stra...,"['dead', 'kills', 'dead', 'died', 'period', 'f...",70,"movies: At least 1 F-word, 4 scatological term..."


In [11]:
references = list(df_predictions['language_summary'])
summaries = list(df_predictions['dolly_8b'])

In [12]:
P, R, F1 = bert_score.score(summaries, references, lang='en')
bertscore_precision = P.mean().item()
bertscore_recall = R.mean().item()
bertscore_f1 = F1.mean().item()

print("BERTScore Precision:", bertscore_precision)
print("BERTScore Recall:", bertscore_recall)
print("BERTScore F1:", bertscore_f1)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore Precision: 0.8119304776191711
BERTScore Recall: 0.8642581105232239
BERTScore F1: 0.8364731669425964
